In [27]:
import getpass
import os

In [28]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [19]:
os.environ["LANGCHAIN_TRACING_V2"] = "True"

api_key = getpass.getpass(prompt= "Enter your api key: ")
os.environ["LANGCHAIN_API_KEY"] = api_key

### Tavily

In [15]:
#loading Tavily api key
os.environ["TAVILY_API_KEY"] = getpass.getpass()
#Importing Tavily
from langchain_community.tools.tavily_search import TavilySearchResults

In [16]:
results = TavilySearchResults(max_results= 2)

In [18]:
results.invoke("who is Keside Onyeocha Olatunbosun?")

[{'url': 'https://www.linkedin.com/posts/keside-onyeocha-bb48b9298_startups-tech-innovation-activity-7199779056173600769-ZN3A',
  'content': 'I&#39;ve been reflecting a lot lately on the choice between building a startup and joining a big tech company. The Lightcone podcast recently provided some…'},
 {'url': 'https://ng.linkedin.com/in/keside-onyeocha-bb48b9298',
  'content': 'Liked by Keside Onyeocha. Founder of Cave.AI Hub || Researcher & Algorithm Development & Artificial intelligence & Natural Language Processing & AI Researcher & Predictive Analytics · Passionate about leveraging machine learning to solve real-world challenges.<br><br>As a skilled Machine Learning Engineer, I specialize in building advanced ...'}]

### Retrieval

In [31]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
doc = loader.load()

documents = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 200 
).split_documents(doc)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()




In [35]:
retriever.invoke("how to upload a dataset?")

[Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";c

In [36]:
from langchain.tools.retriever import create_retriever_tool

In [37]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [39]:
tools = [results, retriever_tool]

In [41]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4")

In [43]:
from langchain_core.messages import HumanMessage
response = model.invoke([HumanMessage(content = "Hi")])
response.content

'Hello! How can I assist you today?'

In [44]:
model_with_tools = model.bind_tools(tools)

In [45]:
response = model.invoke([HumanMessage(content = "Hi")])
print(f"Content String: {response.content}")
print(f"toolcalls: {response.tool_calls}")

Content String: Hello! How can I assist you today?
toolcalls: []


In [47]:
response = model_with_tools.invoke([HumanMessage(content="How much is an regular sized apartment in newyork?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'average price of an apartment in New York'}, 'id': 'call_N0T83RVrOYRFQnQYqnYp8Jod'}]


### Creating the Agent

In [51]:
#create_tool_calling_executor will call .bind_tools for us under the hood.
from langgraph.prebuilt import chat_agent_executor
agent_executor = chat_agent_executor.create_tool_calling_executor(model, tools)

In [55]:
response = agent_executor.invoke({"messages": [HumanMessage(content = "Hi!?")]})
response

{'messages': [HumanMessage(content='Hi!?', id='7079d83d-8163-44c0-bf75-55d23bb41f1c'),
  AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 129, 'total_tokens': 139}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-79c8041b-6e33-40ff-9451-5de889462bc7-0')]}

In [58]:
#lets retrieve
response = agent_executor.invoke(
    {"messages": [HumanMessage(content = "how can langsmith help with testing?")]}
)
response

{'messages': [HumanMessage(content='how can langsmith help with testing?', id='9270a3cd-88e3-43b3-ad7d-a464ad21766b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zzs7JmFdkSXXVo3pTQzoA8Ad', 'function': {'arguments': '{\n  "query": "how can LangSmith help with testing"\n}', 'name': 'langsmith_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 135, 'total_tokens': 157}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-59fdd717-dfd2-4edf-9992-d7b3c94dc281-0', tool_calls=[{'name': 'langsmith_search', 'args': {'query': 'how can LangSmith help with testing'}, 'id': 'call_zzs7JmFdkSXXVo3pTQzoA8Ad'}]),
  ToolMessage(content='Get started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmi

In [57]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="how can langsmith help with testing?")]}
)
response["messages"]

[HumanMessage(content='how can langsmith help with testing?', id='8534c119-76c1-4087-a5a5-c49f563e1b22'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sQ9pBCM8mTe3J0zmlrq9ZWay', 'function': {'arguments': '{\n  "query": "how can LangSmith help with testing"\n}', 'name': 'langsmith_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 135, 'total_tokens': 157}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6f91b136-8e56-461a-9bdc-f15803cf9e23-0', tool_calls=[{'name': 'langsmith_search', 'args': {'query': 'how can LangSmith help with testing'}, 'id': 'call_sQ9pBCM8mTe3J0zmlrq9ZWay'}]),
 ToolMessage(content='Get started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is 

In [60]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content= "what is the weather in Lagos, Nigeria")]}
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JmrD27nT8qqUATp07PZ6fLRn', 'function': {'arguments': '{\n  "query": "current weather in Lagos Nigeria"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ac6bdddd-0e76-4ac1-894b-35024959f56c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Lagos Nigeria'}, 'id': 'call_JmrD27nT8qqUATp07PZ6fLRn'}])]}}
---
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Lagos\', \'region\': \'Lagos\', \'country\': \'Nigeria\', \'lat\': 6.45, \'lon\': 3.4, \'tz_id\': \'Africa/Lagos\', \'localtime_epoch\': 1716905797, \'localtime\': \'2024-05-28 15:16\'}, \'current\': {\'l

In [62]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="what is the weather in Lagos, Nigeria")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in Lagos, Nigeria'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Lagos', 'region': 'Lagos', 'country': 'Nigeria', 'lat': 6.45, 'lon': 3.4, 'tz_id': 'Africa/Lagos', 'localtime_epoch': 1716906241, 'localtime': '2024-05-28 15:24'}, 'current': {'last_updated_epoch': 1716905700, 'last_updated': '2024-05-28 15:15', 'temp_c': 32.0, 'temp_f': 89.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 230, 'wind_dir': 'SW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 75, 'cloud': 50, 'feelslike_c': 40.5, 'feelslike_f': 104.9, 'windchill_c': 29.9, 'windchill_f': 85.7, 'heatindex_c': 35.0, 'heatindex_f': 94.9, 'dewpoint_c': 24.1, 'dewpoint_f': 75.4, 'vis_km': 

In [63]:
#since the Agent is Stateless to add memory we add in a checkpoint using tread_id
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")

In [64]:
agent_executor = chat_agent_executor.create_tool_calling_executor(
    model, tools, checkpointer = memory
)
config = {"configurable": {"thread_id": "abc123"}}

In [65]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content = "Hi, i am keside!")]}, config
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='Hello Keside! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 134, 'total_tokens': 146}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-edb9bf82-a666-418b-8ce9-8f333b8a0704-0')]}}
---


In [68]:
#to check if it stores value
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content = "do you remember my name?")]}, config
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='Yes, I do remember. Your name is Keside. How can I assist you further?', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 159, 'total_tokens': 179}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b9673b1c-02c9-4d6e-8176-980989796d1e-0')]}}
---


In [23]:
!pip install langchain-openai

     ------------------------------------ 320.6/320.6 kB 620.8 kB/s eta 0:00:00
     -------------------------------------- 798.9/798.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.5.2
    Uninstalling tiktoken-0.5.2:
      Successfully uninstalled tiktoken-0.5.2
  Attempting uninstall: openai
    Found existing installation: openai 1.3.4
    Uninstalling openai-1.3.4:
      Successfully uninstalled openai-1.3.4


In [30]:
!pip install faiss-cpu

     ---------------------------------------- 14.5/14.5 MB 1.3 MB/s eta 0:00:00
